In [1]:
import pandas as pd
import numpy as np
import os

#os - operating system dependent. to read/write files
#pd - pandas for data analysis/modeling

data_path = 'data/'
dataframes_path = 'dataframes/'

#get DataFrames
#Reformated sales into one dataframe in 2.1_CreateDailySales
trainingDaySet    = pd.read_pickle(os.path.join(dataframes_path, 'trainingDaySet.pickle'))

In [2]:
trainingDaySet.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-01-02,0,59,22154,999.00,1,37
1,2013-01-03,0,25,2552,899.00,1,58
2,2013-01-05,0,25,2552,899.00,-1,58
3,2013-01-06,0,25,2554,1709.05,1,58
4,2013-01-15,0,25,2555,1099.00,1,56


In [3]:
trainingDaySet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
date                datetime64[ns]
date_block_num      int8
shop_id             int8
item_id             int16
item_price          float64
item_cnt_day        int16
item_category_id    int8
dtypes: datetime64[ns](1), float64(1), int16(2), int8(3)
memory usage: 86.8 MB


In [4]:
trainingDaySet['date'] = trainingDaySet.copy().set_index('date').index.to_period('M').astype(str)
trainingDaySet['date']=pd.to_datetime(trainingDaySet['date'])

In [5]:
trainingDaySet.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,2013-01-01,0,59,22154,999.00,1,37
1,2013-01-01,0,25,2552,899.00,1,58
2,2013-01-01,0,25,2552,899.00,-1,58
3,2013-01-01,0,25,2554,1709.05,1,58
4,2013-01-01,0,25,2555,1099.00,1,56


In [6]:
trainingDaySet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
date                datetime64[ns]
date_block_num      int8
shop_id             int8
item_id             int16
item_price          float64
item_cnt_day        int16
item_category_id    int8
dtypes: datetime64[ns](1), float64(1), int16(2), int8(3)
memory usage: 86.8 MB


In [7]:
# Groupby data to get shop-item-month
_trainingMonthSet = trainingDaySet.copy().groupby(['date_block_num','shop_id', 'item_id'],as_index=False).agg({'item_price':np.mean,'item_cnt_day':'sum'})

_trainingMonthSet.columns = [col if col!='item_cnt_day' else 'item_cnt_month' for col in _trainingMonthSet.columns.values] 
_trainingMonthSet.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,0,32,221.0,6
1,0,0,33,347.0,3
2,0,0,35,247.0,1
3,0,0,43,221.0,1
4,0,0,51,128.5,2


In [8]:
_trainingMonthSet.shape

(1609124, 5)

In [9]:
trainingMonthSet=pd.merge(trainingDaySet[['shop_id', 'item_id','date_block_num','date','item_category_id']],_trainingMonthSet,how='left', on=['date_block_num','shop_id', 'item_id'])
trainingMonthSet.head()

,shop_id,item_id,date_block_num,date,item_category_id,item_price,item_cnt_month
0,59,22154,0,2013-01-01,37,999.00,1
1,25,2552,0,2013-01-01,58,899.00,0
2,25,2552,0,2013-01-01,58,899.00,0
3,25,2554,0,2013-01-01,58,1709.05,1
4,25,2555,0,2013-01-01,56,1099.00,1


In [10]:
trainingMonthSet.shape

(2935849, 7)

In [11]:
trainingMonthSet=trainingMonthSet.drop_duplicates(['date_block_num','shop_id', 'item_id'])
trainingMonthSet.head()

,shop_id,item_id,date_block_num,date,item_category_id,item_price,item_cnt_month
0,59,22154,0,2013-01-01,37,999.00,1
1,25,2552,0,2013-01-01,58,899.00,0
3,25,2554,0,2013-01-01,58,1709.05,1
4,25,2555,0,2013-01-01,56,1099.00,1
5,25,2564,0,2013-01-01,59,349.00,1


In [12]:
trainingMonthSet.shape

(1609124, 7)

In [13]:
trainingMonthSet.to_pickle(dataframes_path+'trainingMonthSet.pickle')